# Data Joins
Joing the multiple CSVs from the GPRA coding project due to the amount of pyton scripts fragmenting the data frame

In [1]:
# importing packages
import pandas as pd
from pandas import ExcelWriter
import datetime as dt
from datetime import datetime

In [2]:
#Importing data
GPRA1 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA1_STAR.csv", parse_dates = ['InterviewDate'])
GPRA2 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA2_STAR.csv")
GPRA3 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA3_STAR.csv")
GPRA4 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA4_STAR.csv")
GPRA5 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA5_STAR.csv")
GPRA6 = pd.read_csv(r"\\cdchomedir01\home1\tyogarro\Jan GPRA\GPRA6_STAR.csv")
client_id = pd.read_excel(r"W:\Research and Evaluation\STAR\GPRA\GPRA Client ID Master List_Full.xlsx",sheet_name = 'List')


In [3]:
#Merging all of the GPRA files
Full_GPRA_File = GPRA1.merge(GPRA2, on = 'ClientID', how = 'left')\
            .merge(GPRA3, on = 'ClientID', how = 'left')\
            .merge(GPRA4, on = 'ClientID', how = 'left')\
            .merge(GPRA5, on = 'ClientID', how = 'left')\
            .merge(GPRA6, on = 'ClientID', how = 'left')

In [4]:
# dropping duplicates
Full_GPRA_File = Full_GPRA_File.drop_duplicates()

In [5]:
#Adding ClientID
client_id = client_id[['PATIENT','GPRA ID']] # <-Subsetting
client_id.rename(columns = {'GPRA ID':'ClientID'}, inplace = True)

In [6]:
#Subsetting dataset for merge to create IntakeDate Column 
Intake_Date_Merge = Full_GPRA_File[['ClientID','InterviewType','InterviewDate']]

#Filtering dataset for intake
Intake_Date_Merge = Intake_Date_Merge[Intake_Date_Merge['InterviewType'] == 1]

#Subsetting dataset further
Intake_Date_Merge = Intake_Date_Merge[['ClientID', 'InterviewDate']]

#Renaming columns
Intake_Date_Merge.rename(columns = {'InterviewDate':'IntakeDate'}, inplace = True)

#Merging files
Full_GPRA_File_STAR = Full_GPRA_File.merge(Intake_Date_Merge, on= 'ClientID', how = 'left')

# Merging file to return client_id
Full_GPRA_File_STAR = Full_GPRA_File_STAR.merge(client_id, on = 'PATIENT', how = 'left') # <- returning ClientIDs

Full_GPRA_File_STAR = Full_GPRA_File_STAR[Full_GPRA_File_STAR['PROVIDER_COUNTY'] != 'Monmouth'] # <- excluding Monmouth County

In [7]:
# QA Provider County
Full_GPRA_File_STAR['PROVIDER_COUNTY'].unique()

array(['Middlesex', 'Essex', 'Hudson', nan], dtype=object)

### Modify Current Upload Date Range

In [8]:
Current_Upload = '_01.13.25-01.19.25_'
## Next Upload: _01.20.25-01.26.25_

In [9]:
# Date Filter for current date range and eliminating non interviews
Full_GPRA_File_STAR = Full_GPRA_File_STAR[(Full_GPRA_File_STAR['InterviewDate']>='2025-01-13') & (Full_GPRA_File_STAR['ConductedInterview'] != 0)]


In [10]:
# QA Interview Types
Full_GPRA_File_STAR['InterviewType'].unique()

array([1., 2.])

In [11]:
# Subsetting file based on Interview Type
STAR_Intakes = Full_GPRA_File_STAR[Full_GPRA_File_STAR['InterviewType']==1]
STAR_Follow_Ups =Full_GPRA_File_STAR[Full_GPRA_File_STAR['InterviewType']==2]
STAR_Discharges = Full_GPRA_File_STAR[Full_GPRA_File_STAR['InterviewType']==5]

In [12]:
# creating file path object
current_upload_intakes ='STAR_BATCH_Upload'+Current_Upload+'Intakes.xlsx'
current_upload_follow_ups ='STAR_BATCH_Upload'+Current_Upload+'Follow-Ups.xlsx'
current_upload_discharges ='STAR_BATCH_Upload'+Current_Upload+'Discharges.xlsx'

In [13]:
# Counties of Current File
STAR_Intakes['PROVIDER_COUNTY'].unique()

array(['Essex', 'Hudson'], dtype=object)

# [ExcelWriter](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.ExcelWriter.html)

Using ExcelWriter to create Excel files with multiple sheets based on the PROVIDER_COUNTY column for different interview types.

In [14]:
# STAR Intakes
intake_writer = ExcelWriter(current_upload_intakes) #--> creating intake excel file
STAR_Intakes[STAR_Intakes['PROVIDER_COUNTY']=='Essex'].to_excel(intake_writer,'Essex_Intakes', index = False)
STAR_Intakes[STAR_Intakes['PROVIDER_COUNTY']=='Hudson'].to_excel(intake_writer,'Hudson_Intakes', index = False)
STAR_Intakes[STAR_Intakes['PROVIDER_COUNTY']=='Middlesex'].to_excel(intake_writer,'Middlesex_Intakes', index = False)
intake_writer.close()


In [15]:
# Counties of Current File
STAR_Follow_Ups['PROVIDER_COUNTY'].unique()

array(['Essex'], dtype=object)

In [16]:
# STAR Follow-Ups
follow_up_writer = ExcelWriter(current_upload_follow_ups) #--> creating follow-up excel file
STAR_Follow_Ups[STAR_Follow_Ups['PROVIDER_COUNTY']=='Essex'].to_excel(follow_up_writer,'Essex_Follow-Ups', index = False)
STAR_Follow_Ups[STAR_Follow_Ups['PROVIDER_COUNTY']== 'Hudson'].to_excel(follow_up_writer,'Hudson_Follow-Ups', index = False)
STAR_Follow_Ups[STAR_Follow_Ups['PROVIDER_COUNTY']== 'Middlesex'].to_excel(follow_up_writer,'Middlesex_Follow-Ups', index = False)
follow_up_writer.close()

In [17]:
# Counties of Current File
STAR_Discharges['PROVIDER_COUNTY'].unique()

array([], dtype=object)

In [18]:
discharge_writer = ExcelWriter(current_upload_discharges) #--> creating follow-up excel file
STAR_Discharges[STAR_Discharges['PROVIDER_COUNTY']=='Essex'].to_excel(discharge_writer,'Essex_Discharges', index = False)
STAR_Discharges[STAR_Discharges['PROVIDER_COUNTY']== 'Hudson'].to_excel(discharge_writer,'Hudson_Discharges', index = False)
STAR_Discharges[STAR_Discharges['PROVIDER_COUNTY']== 'Middlesex'].to_excel(discharge_writer,'Middlesex_Discharges', index = False)
discharge_writer.close()